# The dataframe with three columns: PostalCode, Borough, and Neighborhood

## Import Libraries

We will need requests for getting the HTML contents of the website and lxml.html for parsing the relevant fields. Finally, we will store the data on a Pandas Dataframe.

In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
from io import StringIO

## Scrape Table Cells

The code below allows us to get Toronto Postal codes data of the HTML table at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

For sanity check, ensure that all the rows have the same width. If not, we probably got something more than just the table.

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Looks like all our rows have exactly 3 columns. This means all the data collected on tr_elements are from the table.

## Parse Table HeaderParse Table Header

Next, let’s parse the first row as our header.

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print '%d:"%s"'%(i,name)
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



## Creating Pandas DataFrame

Each header is appended to a tuple along with an empty list.

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Just to be sure, let’s check the length of each column. Ideally, they should all be the same.

In [6]:
[len(C) for (title,C) in col]

[287, 287, 287]

This shows that each of our 3 columns has exactly 287 values.

Now we are ready to create the DataFrame:

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

## Data Wrangling

In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
       df.shape[0]
    )
)

The dataframe has 11 boroughs and 287 neighborhoods.


In [9]:
#Let's save the postal codes
df.to_csv('First_df.csv')

In [10]:
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')

In [11]:
#Let's rename Postcode to PostalCode
df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
df.rename(columns={"Neighbourhood\n": "Neighborhood"}, inplace=True)

In [12]:
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [13]:
# Let's take the trailing newlines and spaces away from data

df['PostalCode'] = df['PostalCode'].str.strip()
df['Borough'] = df['Borough'].str.strip()
df['Neighborhood'] = df['Neighborhood'].str.strip()

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [15]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

So far so good


In [16]:
#Lets' drop all cells with a borough that is Not assigned and reset index
df.drop(df[df['Borough']  == 'Not assigned'].index, inplace = True) 
df = df.sort_values('PostalCode', ascending=True)
df.reset_index()

,index,PostalCode,Borough,Neighborhood
0,10,M1B,Scarborough,Rouge
1,11,M1B,Scarborough,Malvern
2,28,M1C,Scarborough,Port Union
3,27,M1C,Scarborough,Rouge Hill
4,26,M1C,Scarborough,Highland Creek
5,41,M1E,Scarborough,Guildwood
6,42,M1E,Scarborough,Morningside
7,43,M1E,Scarborough,West Hill
8,52,M1G,Scarborough,Woburn
9,61,M1H,Scarborough,Cedarbrae


In [17]:
df.head()

,PostalCode,Borough,Neighborhood
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
28,M1C,Scarborough,Port Union
27,M1C,Scarborough,Rouge Hill
26,M1C,Scarborough,Highland Creek


In [18]:
df.shape

(210, 3)

In [19]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

for ind in df.index: 
    if df['Neighborhood'][ind] == 'Not assigned':
        df['Neighborhood'][ind]= df['Borough'][ind]

In [20]:
df

,PostalCode,Borough,Neighborhood
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
28,M1C,Scarborough,Port Union
27,M1C,Scarborough,Rouge Hill
26,M1C,Scarborough,Highland Creek
41,M1E,Scarborough,Guildwood
42,M1E,Scarborough,Morningside
43,M1E,Scarborough,West Hill
52,M1G,Scarborough,Woburn
61,M1H,Scarborough,Cedarbrae


In [21]:
#How many unique postal codes there are in the duplicates list?
duplicates = df[df.duplicated(['PostalCode'])]
unique_list = []     
unique_list=duplicates.PostalCode.unique()

In [22]:
duplicates

,PostalCode,Borough,Neighborhood
11,M1B,Scarborough,Malvern
27,M1C,Scarborough,Rouge Hill
26,M1C,Scarborough,Highland Creek
42,M1E,Scarborough,Morningside
43,M1E,Scarborough,West Hill
91,M1K,Scarborough,Ionview
92,M1K,Scarborough,Kennedy Park
108,M1L,Scarborough,Oakridge
106,M1L,Scarborough,Clairlea
124,M1M,Scarborough,Scarborough Village West


In [23]:
len(unique_list)

56

In [24]:
gk = df.groupby('PostalCode') 
  
# Let's print the first entries in all the groups formed. 
result=gk.first()
result
for i in result.index:
      print(result)

                     Borough  \
PostalCode                     
M1B              Scarborough   
M1C              Scarborough   
M1E              Scarborough   
M1G              Scarborough   
M1H              Scarborough   
M1J              Scarborough   
M1K              Scarborough   
M1L              Scarborough   
M1M              Scarborough   
M1N              Scarborough   
M1P              Scarborough   
M1R              Scarborough   
M1S              Scarborough   
M1T              Scarborough   
M1V              Scarborough   
M1W              Scarborough   
M1X              Scarborough   
M2H               North York   
M2J               North York   
M2K               North York   
M2L               North York   
M2M               North York   
M2N               North York   
M2P               North York   
M2R               North York   
M3A               North York   
M3B               North York   
M3C               North York   
M3H               North York   
M3J     

In [25]:
#Apply groupby and do the concatenating. More than one neighborhood can exist in one postal code area and they need to be combined into one row with the neighborhoods separated with a comma
df['Neighborhood'] = df[['PostalCode', 'Borough','Neighborhood']].groupby(['PostalCode'])['Neighborhood'].transform(lambda x: ','.join(x))
df[['PostalCode','Borough','Neighborhood']].drop_duplicates()

,PostalCode,Borough,Neighborhood
10,M1B,Scarborough,"Rouge,Malvern"
28,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
41,M1E,Scarborough,"Guildwood,Morningside,West Hill"
52,M1G,Scarborough,Woburn
61,M1H,Scarborough,Cedarbrae
75,M1J,Scarborough,Scarborough Village
90,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
107,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
122,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
140,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [26]:
#Sorting the dataframe by postal code and dropping duplicates
df = df.sort_values('PostalCode', ascending=True)
df = df.drop_duplicates(subset='PostalCode', keep='first')
df

,PostalCode,Borough,Neighborhood
10,M1B,Scarborough,"Rouge,Malvern"
28,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
41,M1E,Scarborough,"Guildwood,Morningside,West Hill"
52,M1G,Scarborough,Woburn
61,M1H,Scarborough,Cedarbrae
75,M1J,Scarborough,Scarborough Village
92,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
107,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
122,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
140,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [27]:
#Let's verify there is no duplicates
duplicates = df[df.duplicated(['PostalCode'])]


In [28]:
duplicates

,PostalCode,Borough,Neighborhood


In [29]:
#Let's reset index and drop the old index
df = df.reset_index()
del df['index']

In [30]:
#Now the dataframe should be sorted by postal code and reindexed
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
8,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [31]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
       df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [32]:
#Let's save the postal codes
df.to_csv('TorontoPostalCodes_df.csv')

In [35]:
#Let's see what is the shape of the dataframe now
df.shape

(103, 3)